# Data Preprocessing – TravelTide Project

The purpose of this notebook is to prepare the raw data so that 
it is suitable for subsequent analysis and modeling.

### 1. Imports & Settings

In [ ]:
# loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 2. Load the Data

In [ ]:
# loading session_base.csv
df_session = pd.read_csv("../data/session_base.csv")
df_session.head()

,session_id,user_id,trip_id,session_start,session_end,page_clicks,flight_discount,flight_discount_amount,hotel_discount,hotel_discount_amount,...,trip_airline,destination_airport_lat,destination_airport_lon,base_fare_usd,hotel_name,nights,rooms,check_in_time,check_out_time,hotel_price_per_room_night_usd
0,474007-040d8694832845e98a710bfc20194f52,474007,NaN,2023-03-28 21:35:00,2023-03-28 21:35:24,3,False,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,491175-456ae210ac9c4150800565740316dc77,491175,NaN,2023-03-28 19:14:00,2023-03-28 19:15:16,10,False,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,520603-67823ebddae848398ce155649a426242,520603,NaN,2023-03-28 16:38:00,2023-03-28 16:38:45,6,False,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,524180-e1e5d53c38114c8bbd236d9f603546ca,524180,NaN,2023-03-28 18:56:00,2023-03-28 18:57:38,13,False,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,527801-5fe15cbf9fee42d5b2c9e76aeef603a1,527801,NaN,2023-03-28 20:03:00,2023-03-28 20:03:29,4,False,NaN,True,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. Initial Data Overview

In [ ]:
# show number of rows and columns
print(df_session.shape)
print()
# show column names
print(df_session.columns)
print()
# show data types of each column
print(df_session.dtypes)


(49211, 41)

Index(['session_id', 'user_id', 'trip_id', 'session_start', 'session_end',
       'page_clicks', 'flight_discount', 'flight_discount_amount',
       'hotel_discount', 'hotel_discount_amount', 'flight_booked',
       'hotel_booked', 'cancellation', 'birthdate', 'gender', 'married',
       'has_children', 'home_country', 'home_city', 'home_airport',
       'home_airport_lat', 'home_airport_lon', 'sign_up_date',
       'origin_airport', 'destination', 'destination_airport', 'seats',
       'return_flight_booked', 'departure_time', 'return_time', 'checked_bags',
       'trip_airline', 'destination_airport_lat', 'destination_airport_lon',
       'base_fare_usd', 'hotel_name', 'nights', 'rooms', 'check_in_time',
       'check_out_time', 'hotel_price_per_room_night_usd'],
      dtype='str')

session_id                            str
user_id                             int64
trip_id                               str
session_start                         str
session_end            

### 4. Data Cleaning
#### 4.1 Handling Duplicate Rows

In [ ]:
n_dups = df_session.duplicated().sum()
n_dups

np.int64(0)

A check for fully duplicated rows showed that no duplicate records are present in the dataset.
Therefore, no rows were removed in this step.

#### 4.2 Handling Missing Values

In [ ]:
# check missing values in each column
print(df_session.isnull().sum().sort_values(ascending=False))

hotel_discount_amount             43006
flight_discount_amount            40929
return_time                       35559
departure_time                    34941
base_fare_usd                     34941
origin_airport                    34941
destination                       34941
return_flight_booked              34941
seats                             34941
destination_airport_lat           34941
trip_airline                      34941
checked_bags                      34941
destination_airport_lon           34941
destination_airport               34941
hotel_name                        34485
nights                            34485
rooms                             34485
check_out_time                    34485
check_in_time                     34485
hotel_price_per_room_night_usd    34485
trip_id                           32509
user_id                               0
session_start                         0
session_id                            0
session_end                           0


The missing value analysis shows that several features contain missing values.
These missing values are mostly booking-dependent (e.g. hotel and return flight information)
and indicate that the corresponding service was not booked.
Therefore, missing values were kept and not imputed at this stage.


#### 4.3 Data Type Conversion

In [ ]:
# formatting datetime columns
datetime_cols = [
    'session_start', 'session_end',
    'departure_time', 'return_time',
    'check_in_time', 'check_out_time',
    'sign_up_date', 'birthdate'
]

for col in datetime_cols:
    df_session[col] = pd.to_datetime(df_session[col], errors='coerce')



Timestamp-related features were converted from string to datetime format
to enable time-based calculations.


#### 4.4 Outlier Detection and Treatment


In [ ]:
df_session.describe()

,user_id,session_start,session_end,page_clicks,flight_discount_amount,hotel_discount_amount,birthdate,home_airport_lat,home_airport_lon,sign_up_date,...,return_time,checked_bags,destination_airport_lat,destination_airport_lon,base_fare_usd,nights,rooms,check_in_time,check_out_time,hotel_price_per_room_night_usd
count,49211.000000,49211,48968,49211.000000,8282.000000,6205.000000,49211,49211.000000,49211.000000,49211,...,13652,14270.000000,14270.000000,14270.000000,14270.000000,14726.000000,14726.000000,12241,14726,14726.000000
mean,545282.694946,2023-03-21 11:25:24.870192,2023-03-21 06:08:18.458932,17.588791,0.139864,0.112192,1982-02-26 19:15:57.371319,38.427222,-94.183762,2023-01-20 00:56:58.357684,...,2023-04-20 02:07:30.659244,0.592922,38.610024,-88.947203,525.901390,3.672484,1.194554,2023-04-07 17:03:18.079070,2023-04-11 04:10:28.276518,177.937390
min,23557.000000,2023-01-04 00:01:00,2023-01-04 00:04:23,1.000000,0.050000,0.050000,1935-05-10 00:00:00,21.316000,-157.927000,2021-07-22 00:00:00,...,2023-01-08 07:00:00,0.000000,-37.008000,-157.927000,2.410000,-2.000000,1.000000,2023-01-07 11:21:26.415000,2023-01-08 11:00:00,17.000000
25%,517119.000000,2023-02-05 22:31:30,2023-02-05 21:23:03.750000,6.000000,0.100000,0.050000,1974-09-08 00:00:00,33.818000,-112.383000,2023-01-08 00:00:00,...,2023-02-19 13:00:00,0.000000,33.942000,-112.383000,200.432500,1.000000,1.000000,2023-02-15 09:37:21.045000,2023-02-19 11:00:00,99.000000
50%,540308.000000,2023-03-09 11:04:00,2023-03-09 03:21:28.500000,13.000000,0.100000,0.100000,1981-10-09 00:00:00,39.103000,-90.035000,2023-01-19 00:00:00,...,2023-03-27 10:00:00,1.000000,39.872000,-87.752000,381.765000,2.000000,1.000000,2023-03-21 14:51:35.010000,2023-03-25 11:00:00,148.000000
75%,573922.000000,2023-04-28 11:23:00,2023-04-27 20:13:15,22.000000,0.200000,0.150000,1989-03-16 00:00:00,42.238000,-79.370000,2023-02-05 00:00:00,...,2023-05-27 12:15:00,1.000000,42.409000,-75.241000,600.632500,5.000000,1.000000,2023-05-17 10:03:19.305000,2023-05-20 11:00:00,222.000000
max,844489.000000,2023-07-28 19:58:52,2023-07-28 20:08:52,566.000000,0.600000,0.450000,2006-12-28 00:00:00,61.251000,-63.499000,2023-05-18 00:00:00,...,2024-07-30 16:00:00,8.000000,55.972000,174.792000,21548.040000,43.000000,4.000000,2024-07-17 00:33:41.625000,2024-07-29 11:00:00,1376.000000
std,64640.047648,NaN,NaN,21.495987,0.083914,0.062119,NaN,6.185962,18.065868,NaN,...,NaN,0.677641,6.949739,33.090373,818.296468,3.806717,0.499344,NaN,NaN,118.334569


Potential outliers in numeric features were inspected.
No outliers were removed, as extreme values are plausible within the travel booking context
(e.g. high prices or long booking lead times).

### 5. Feature Engineering

New features were created to better capture user behavior and booking characteristics.
These engineered features improve interpretability and provide more meaningful inputs for subsequent analysis and modeling.


In [ ]:
# calculate session duration in seconds in a new column
df_session['session_duration'] = (df_session['session_end'] - df_session['session_start']).dt.total_seconds()

In [ ]:
# calculate booking lead time in days in a new column
df_session['booking_lead_time_days'] = (
    df_session['departure_time'] - df_session['session_start']
).dt.days

New features were created to better capture user behavior and booking characteristics.
Only features relevant for subsequent analysis and clustering were retained.


### 6. Final Dataset


In [ ]:
# show new dataset
df_session.shape
df_session.head()



,session_id,user_id,trip_id,session_start,session_end,page_clicks,flight_discount,flight_discount_amount,hotel_discount,hotel_discount_amount,...,destination_airport_lon,base_fare_usd,hotel_name,nights,rooms,check_in_time,check_out_time,hotel_price_per_room_night_usd,session_duration,booking_lead_time_days
0,474007-040d8694832845e98a710bfc20194f52,474007,NaN,2023-03-28 21:35:00,2023-03-28 21:35:24,3,False,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,24.0,NaN
1,491175-456ae210ac9c4150800565740316dc77,491175,NaN,2023-03-28 19:14:00,2023-03-28 19:15:16,10,False,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,76.0,NaN
2,520603-67823ebddae848398ce155649a426242,520603,NaN,2023-03-28 16:38:00,2023-03-28 16:38:45,6,False,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,45.0,NaN
3,524180-e1e5d53c38114c8bbd236d9f603546ca,524180,NaN,2023-03-28 18:56:00,2023-03-28 18:57:38,13,False,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,98.0,NaN
4,527801-5fe15cbf9fee42d5b2c9e76aeef603a1,527801,NaN,2023-03-28 20:03:00,2023-03-28 20:03:29,4,False,NaN,True,0.2,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,29.0,NaN


In [ ]:
# new csv file with preprocessed data
df_session.to_csv("../data/"traveltide_preprocessed.csv", index=False)

## Summary
- Checked for duplicate records
- Analyzed and interpreted missing values
- Corrected data types
- Created new features
- Dataset is ready for analysis and modeling
